In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy

In [8]:
CWD = os.getcwd()

In [30]:
aapl_csv_path = os.path.join(CWD, 'Deeplearning-Finance', 'AAPL.csv')
df = pd.read_csv(aapl_csv_path)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400


# Using past 60 days of data to predict next day's data
> Date will be dropped for now since putting the days in sequence likely captures the 'Date' information

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Assuming you already have your data loaded in 'df' (with columns: 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume')
# df = pd.read_csv('your_data.csv')

# Step 1: Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].values)

# Step 2: Create sequences for LSTM input (e.g., use 60 previous days to predict the next day's values)
def create_sequences(data, seq_length=60):
    x_data = []
    y_data = []
    
    for i in range(seq_length, len(data)):
        x_data.append(data[i-seq_length:i])
        y_data.append(data[i])
        
    return np.array(x_data), np.array(y_data)

# Create sequences
X, y = create_sequences(scaled_data, seq_length=60)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 4: Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=6))

# Step 5: Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 6: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 7: Predict the next day's values for a specific test point
# Select a random index from the test set (you can choose any index here)
index = 100  # You can select any index in the test set
test_input = X_test[index].reshape(1, X_test.shape[1], X_test.shape[2])

# Predict the values for the next day
predicted_values = model.predict(test_input)

# Inverse transform the prediction and the actual value
predicted_values = scaler.inverse_transform(predicted_values)
actual_values = scaler.inverse_transform(y_test[index].reshape(1, -1))

# Step 8: Print the actual and predicted values
print("Predicted Values (Next Day):")
print(f"Open: {predicted_values[0][0]}")
print(f"High: {predicted_values[0][1]}")
print(f"Low: {predicted_values[0][2]}")
print(f"Close: {predicted_values[0][3]}")
print(f"Adj Close: {predicted_values[0][4]}")
print(f"Volume: {predicted_values[0][5]}")

print("\nActual Values (Next Day):")
print(f"Open: {actual_values[0][0]}")
print(f"High: {actual_values[0][1]}")
print(f"Low: {actual_values[0][2]}")
print(f"Close: {actual_values[0][3]}")
print(f"Adj Close: {actual_values[0][4]}")
print(f"Volume: {actual_values[0][5]}")


Epoch 1/10


c:\Users\yanka\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


247/247 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 6.1460e-04
Epoch 2/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 1.6221e-04
Epoch 3/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 1.6027e-04
Epoch 4/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 1.4605e-04
Epoch 5/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 1.6204e-04
Epoch 6/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 1.3545e-04
Epoch 7/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 1.5876e-04
Epoch 8/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - loss: 1.2949e-04
Epoch 9/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 1.4931e-04
Epoch 10/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 1.3676e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
Predicted Values (Next Day):
Open: 89.23247528076172
High: 89.68207550048828
Low: 87.51905822753906
Close: 88.84752655029297
Adj Close: 77.25399017333984
Volume: 150438208.0

Actual Values (Next Day):
Open: 88.77714538574219
High: 89.50714111328125
L

# Predict whether next day's high would be higher than open

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Assuming you already have your data loaded in 'df' (with columns: 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume')
# df = pd.read_csv('your_data.csv')

# Step 1: Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].values)

# Step 2: Create sequences for LSTM input (e.g., use 60 previous days to predict the next day's values)
def create_sequences(data, seq_length=60):
    x_data = []
    y_data = []
    
    for i in range(seq_length, len(data)):
        x_data.append(data[i-seq_length:i])
        
        # Target: 1 if High > Open for the next day, else 0
        if data[i][1] > data[i][0]:  # High is at index 1 and Open is at index 0
            y_data.append(1)
        else:
            y_data.append(0)
        
    return np.array(x_data), np.array(y_data)

# Create sequences
X, y = create_sequences(scaled_data, seq_length=60)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 4: Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1, activation='sigmoid'))  # Binary classification output

# Step 5: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 6: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 7: Predict the result for a specific test point
index = 100  # You can select any index in the test set
test_input = X_test[index].reshape(1, X_test.shape[1], X_test.shape[2])

# Predict whether the High will be greater than the Open (1 or 0)
predicted_class = model.predict(test_input)

# Convert prediction to binary outcome
predicted_class = (predicted_class > 0.5).astype(int)

# Step 8: Print the actual and predicted values
actual_class = y_test[index]

print(f"Prediction for the test point (High > Open): {'Yes' if predicted_class == 1 else 'No'}")
print(f"Actual result for the test point (High > Open): {'Yes' if actual_class == 1 else 'No'}")


Epoch 1/10


c:\Users\yanka\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


247/247 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.6342 - loss: 0.6592
Epoch 2/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6503 - loss: 0.6462
Epoch 3/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6547 - loss: 0.6431
Epoch 4/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6506 - loss: 0.6470
Epoch 5/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6455 - loss: 0.6491
Epoch 6/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6566 - loss: 0.6412
Epoch 7/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.6499 - loss: 0.6470
Epoch 8/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.6600 - loss: 0.6409
Epoch 9/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.6467 - loss: 0.6470
Epoch 10/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.6517 - loss: 0.6447
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 478ms/step
Prediction for the test point (High > Open): No
Actual result for the test point (High > Open): No


In [21]:
false_predictions = []
num_test = 100

for index in range(num_test):
    # Step 7: Predict the result for a specific test point
    test_input = X_test[index].reshape(1, X_test.shape[1], X_test.shape[2])

    # Predict whether the High will be greater than the Open (1 or 0)
    predicted_class = model.predict(test_input, verbose=0)

    # Convert prediction to binary outcome
    predicted_class = (predicted_class > 0.5).astype(int)

    # Step 8: Print the actual and predicted values
    actual_class = y_test[index]

    if predicted_class != actual_class:
        false_predictions.append(index)

    # print(f"Prediction for the test point (High > Open): {'Yes' if predicted_class == 1 else 'No'}")
    # print(f"Actual result for the test point (High > Open): {'Yes' if actual_class == 1 else 'No'}")

print(f'False Predictions:{len(false_predictions)}/{num_test}')

False Predictions:31/100


# Predict High, Low
> (can ignore this, some error with dataset, likely coz of scaling)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Assuming you already have your data loaded in 'df' (with columns: 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume')
# df = pd.read_csv('your_data.csv')

# Step 1: Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].values)

# Step 2: Create sequences for LSTM input (e.g., use 60 previous days to predict the next day's High and Low)
def create_sequences(data, seq_length=60):
    x_data = []
    y_data = []
    
    for i in range(seq_length, len(data)):
        x_data.append(data[i-seq_length:i])
        # Target: We predict the next day's High and Low
        y_data.append([data[i][1], data[i][2]])  # High is at index 1 and Low is at index 2
        
    return np.array(x_data), np.array(y_data)

# Create sequences
X, y = create_sequences(scaled_data, seq_length=60)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Step 4: Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=2))  # Output layer with 2 units for High and Low

# Step 5: Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 6: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 7: Predict the High and Low values for a specific test point
index = 100  # You can select any index in the test set
test_input = X_test[index].reshape(1, X_test.shape[1], X_test.shape[2])

# Predict the next day's High and Low values
predicted_values = model.predict(test_input)

# Inverse transform the prediction and the actual values
predicted_values = scaler.inverse_transform(np.hstack((np.zeros((predicted_values.shape[0], 4)), predicted_values)))[:, 4:]  # Get only High and Low
actual_values = scaler.inverse_transform(np.hstack((np.zeros((y_test[index].reshape(1, -1).shape[0], 4)), y_test[index].reshape(1, -1))))[:, 4:]

# Step 8: Print the actual and predicted values
print(f"Predicted High: {predicted_values[0][0]}")
print(f"Predicted Low: {predicted_values[0][1]}")

print(f"Actual High: {actual_values[0][0]}")
print(f"Actual Low: {actual_values[0][1]}")


Epoch 1/10


c:\Users\yanka\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 1.8366e-04
Epoch 2/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 4.4640e-06
Epoch 3/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 4.2704e-06
Epoch 4/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 4.3554e-06
Epoch 5/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 4.5171e-06
Epoch 6/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 3.0907e-06
Epoch 7/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - loss: 3.3790e-06
Epoch 8/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 3.4899e-06
Epoch 9/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 2.3837e-06
Epoch 10/10
247/247 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 2.5640e-06
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
Predicted High: 91.08810595068343
Predicted Low: 507663357.3704183
Actual High: 89.2986754069105
Actual Low: 499987975.81432194


# Predict High, Low
> Using predictions, if actual open is within predicted high and low, take it as buy the stock. 
> Sell it if actual low is lower than predicted low because you would sell it at predicted low for a loss. Else, sell it if actual high is higher than predicted high for a profit.
> Else sell it at actual close for a net gain/loss
> Honestly a shit algorithm coz it doesnt predict whether it will go up or down first. But somehow it works. I think about this more tmr :)

In [36]:
# Calculate the index to split the data (80% training, 20% testing)
split_index = int(len(df) * 0.8)

# Training data (first 80% of the data)
train_data = df[:split_index]

# Test data (last 20% of the data)
test_data = df[split_index:]

# Step 1: Prepare the data (Do not scale, use raw data)
# Use 'Open', 'High', 'Low', 'Close', 'Adj Close' as input features for both training and testing sets
train_data = train_data[['Open', 'High', 'Low', 'Close', 'Adj Close']].values
test_data = test_data[['Open', 'High', 'Low', 'Close', 'Adj Close']].values

# Step 2: Create sequences for LSTM input (e.g., use 60 previous days to predict the next day's High and Low)
def create_sequences(data, seq_length=30):
    x_data = []
    y_data = []
    
    for i in range(seq_length, len(data)):
        x_data.append(data[i-seq_length:i])
        # Target: We predict the next day's High and Low
        y_data.append([data[i][1], data[i][2]])  # High is at index 1 and Low is at index 2
        
    return np.array(x_data), np.array(y_data)

# Create sequences for training and testing
X_train, y_train = create_sequences(train_data)
X_test, y_test = create_sequences(test_data)

# Step 3: Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=2))  # Output layer with 2 units for High and Low

# Step 4: Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 5: Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Step 6: Make predictions on the test set
predicted_values = model.predict(X_test)

# Step 7: Simulate trading based on the predictions
total_profit = 0
losses = []
gains = []
closes = []
total = []

for i in range(len(predicted_values)):
    actual_open = test_data[i + 30][0]  # Open value of the current day (start of the next day for prediction)
    predicted_high = predicted_values[i][0]
    predicted_low = predicted_values[i][1]
    actual_low = test_data[i + 30][2]
    actual_high = test_data[i + 30][1]
    actual_close = test_data[i + 30][3]
    
    # Case 1: If Open is within the predicted High and Low, we "buy"
    if predicted_low <= actual_open <= predicted_high:
        # Buy condition: Take the difference between actual open and predicted low (loss) or predicted high (gain)
        
        # If the actual low is lower than predicted low, loss is the difference between Open and predicted low
        if actual_low < predicted_low:
            loss = predicted_low - actual_open
            total_profit += loss
            losses.append(loss)
        
        # If predicted high is higher than actual high, gain is the difference between predicted high and Open
        elif predicted_high > actual_high:
            abs_gain = predicted_high - actual_open
            total_profit += abs_gain
            gains.append(abs_gain)
        
        # If no gain or loss, sell at Close (gain or loss is the difference between Open and Close)
        else:
            net = actual_close - actual_open
            total_profit += net
            closes.append(net)

    total.append(total_profit)
    
    # Print the profit/loss for each prediction if you want to track it
    print(f"Prediction {i + 1}:")
    print(f"Predicted High: {predicted_high}, Predicted Low: {predicted_low}")
    print(f"Actual High: {actual_high}, Actual Low: {actual_low}, Actual Close: {actual_close}")
    print(f"Total Profit/Loss so far: {total_profit}\n")

# Final profit or loss
print(f"Total Profit/Loss after testing: {total_profit}")


Epoch 1/100


c:\Users\yanka\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 199.4839
Epoch 2/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 105.2027
Epoch 3/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 69.5263
Epoch 4/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 53.1126
Epoch 5/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 35.7974
Epoch 6/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 25.9706
Epoch 7/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 22.3674
Epoch 8/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 13.7337
Epoch 9/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 11.6571
Epoch 10/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 11.3776
Epoch 11/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 10.0814
Epoch 12/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 10.0463
Epoch 13/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 9.6067
Epoch 14/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 6.7217
Epoch 15/100
247/247 ━━━━━━━━━━━━━━━

In [59]:
### For closes, find number of closes that result in profit/loss
count_above_zero = sum(1 for x in closes if x > 0)
count_below_zero = len(closes) - count_above_zero

total_days_traded = sum([len(gains), len(losses), len(closes)])
print(f'Days traded: {total_days_traded}')
print(f'Days trade not taken due to actual open being outside of predicted range: {len(predicted_values)-total_days_traded}')
print()

print(f'Days of gains: {len(gains)} + {count_above_zero}(From closing) = {len(gains)+count_above_zero}')
print(f'Days of losses: {len(losses)} + {count_below_zero}(From closing) = {len(losses)+count_below_zero}')
print()

print(f'Total profit from gains = {sum(gains)}')
print(f'Total profit from losses = {sum(losses)}')
print(f'Total net change from closes = {sum(closes)}')
print()

print(f'Grand Total: {total_profit}')

Days traded: 258
Days trade not taken due to actual open being outside of predicted range: 1694

Days of gains: 47 + 68(From closing) = 115
Days of losses: 130 + 13(From closing) = 143

Total profit from gains = 36.15910720825196
Total profit from losses = -48.5892105102539
Total net change from closes = 55.081455230712876

Grand Total: 42.65135192871094
